# Production ready crosslight detection

### Lets create our model and load the weights

In [ ]:
import display

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

from picamera import PiCamera, array
from time import sleep
from numpy import around

In [ ]:
image_height = 256
image_width = 256

In [ ]:
model = Sequential()

model.add(Conv2D(16, (7, 7), activation='relu', padding='same', name='block1_conv1', input_shape=(image_height, image_width, 1)))
model.add(Conv2D(16, (7, 7), activation='relu', padding='same', name='block1_conv2',))
model.add(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), name='block1_pool'))

model.add(Conv2D(32, (5, 5), activation='relu', padding='same', name='block2_conv1',))
model.add(Conv2D(32, (5, 5), activation='relu', padding='same', name='block2_conv2',))
model.add(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), name='block2_pool'))

model.add(Conv2D(64, (3, 3), activation='relu', padding='same', name='block3_conv1',))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', name='block3_conv2',))
model.add(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), name='block3_pool'))

model.add(Flatten())
model.add(Dense(128, activation='relu', name="fc-1"))
model.add(Dropout(1))
model.add(Dense(128, activation='relu', name="fc-2"))
model.add(Dense(6, activation='sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.load_weights('/home/pi/repositories/pren/models/ciffer.h5')

### Raspberry camera logic

In [ ]:
def checkPrediction(prediction):
    if prediction[0] >= 0.90:
        display.one()
        print("1")
        break

    if prediction[1] >= 0.90:
        display.two()
        print("2")
        break

    if prediction[2] >= 0.90:
        display.three()
        print("3")
        break

    if prediction[3] >= 0.90:
        display.four()
        print("4")
        break

    if prediction[4] >= 0.90:
        display.five()
        print("5")
        break

In [ ]:
with PiCamera() as camera:
    camera.resolution = (256, 256)
    sleep(1)
    number_found = False
    
    
    display.one()
    #MAYBE YUV TAKE THE Y
    with array.PiRGBArray(camera) as output:
        while True:
            camera.capture(output, 'rgb')
            
            # Lösungsweg 1
            x = output.array
            x = x.reshape((1,) + x.shape)
            x = x * (1./255)
            
            prediction = around(model.predict(x), 3)[0][0]

            # Lösungsweg 2
            #prediction = around(model.predict(output.array), 3)[0]
            
            print(prediction)
            checkPrediction(prediction)
            
            output.truncate(0)